In [8]:
import numpy as np

np.random.seed(37)

N = 1000

x1 = np.random.normal(1, 1, N)
x2 = np.random.normal(1 + 3.5 * x1, 1, N)
x3 = np.random.normal(2, 1, N)
x4 = np.random.normal(3.8 - 2.5 * x3, 1, N)

data = np.vstack([x1, x2, x3, x4]).T
means = data.mean(axis=0)
mins = data.min(axis=0)
maxs = data.max(axis=0)
cov = np.cov(data.T)
std = np.sqrt(np.diag(cov))
cor = np.corrcoef(data.T)

print('means')
print(means)
print('')
print('mins')
print(mins)
print('')
print('maxs')
print(maxs)
print('')
print('cov')
print(cov)
print('')
print('stddev')
print(std)
print('')
print('correlation matrix')
print(cor)

means
[ 1.01277839  4.52863965  1.98990172 -1.17391554]

mins
[-2.31079823 -8.33142158 -1.80414214 -8.74552729]

maxs
[ 3.92919388 17.57679341  4.82528779  9.27678294]

cov
[[ 9.63461496e-01  3.36840170e+00 -1.12846545e-02 -5.12464592e-02]
 [ 3.36840170e+00  1.27550651e+01 -9.26050108e-02 -8.56265759e-02]
 [-1.12846545e-02 -9.26050108e-02  9.70507183e-01 -2.46328945e+00]
 [-5.12464592e-02 -8.56265759e-02 -2.46328945e+00  7.25484316e+00]]

stddev
[0.98156075 3.5714234  0.98514323 2.69348161]

correlation matrix
[[ 1.          0.9608716  -0.01167002 -0.01938352]
 [ 0.9608716   1.         -0.02632048 -0.0089013 ]
 [-0.01167002 -0.02632048  1.         -0.9283293 ]
 [-0.01938352 -0.0089013  -0.9283293   1.        ]]


In [10]:
from scipy.stats import norm

def get_index_2(index_1, N):
    return [i for i in range(N) if i not in index_1]

def partition_means(index_1, means, index_2=None):
    index_2 = get_index_2(index_1, len(means)) if index_2 is None else index_2
    m_1, m_2 = means[index_1], means[index_2]
    return m_1, m_2

def partition_cov(index_1, cov, index_2=None):
    index_2 = get_index_2(index_1, cov.shape[1]) if index_2 is None else index_2
    s_11 = cov[index_1][:, index_1]
    s_12 = cov[index_1][:, index_2]
    s_21 = cov[index_2][:, index_1]
    s_22 = cov[index_2][:, index_2]

    return s_11, s_12, s_21, np.linalg.inv(s_22)

def partition_x(index_1, x, index_2=None):
    index_2 = get_index_2(index_1, len(x)) if index_2 is None else index_2
    x_1 = x[index_1]
    x_2 = x[index_2]
    return x_1, x_2

def get_log_proba(index_1, data, means, cov, index_2=None, zero=0.000001):
    m_1, m_2 = partition_means(index_1, means, index_2)
    s_11, s_12, s_21, s_22 = partition_cov(index_1, cov, index_2)
    s = (s_11 - s_12.dot(s_22).dot(s_21))[0, 0]

    log_proba = []
    for x in data:
        x_1, x_2 = partition_x(index_1, x, index_2)
        m = (m_1 + s_12.dot(s_22).dot((x_2 - m_2).T))[0]
        p = norm.pdf(x_1, loc=m, scale=s)
        log_p = np.log(p) if p >= zero else 0.0
        log_proba.append(log_p)

    return sum(log_proba)[0], s_12.dot(s_22)[0]

In [12]:
get_log_proba([1], data, means, cov, [0])

(-1407.7504048724827, array([3.49614563]))

In [14]:
get_log_proba([0], data, means, cov, [1])

(-1844.8158517246407, array([0.26408346]))

In [16]:
get_log_proba([1], data, means, cov, [2])

(-3503.3620421459736, array([-0.09541919]))

In [18]:
get_log_proba([1], data, means, cov, [3])

(-3503.951768440483, array([-0.01180268]))

In [20]:
get_log_proba([1], data, means, cov, [0, 2])

(-1406.293970788457, array([ 3.49550407, -0.05477492]))

In [22]:
get_log_proba([1], data, means, cov, [0, 3])

(-1407.1468441245956, array([3.49683168, 0.0128981 ]))

In [24]:
get_log_proba([1], data, means, cov, [2, 3])

(-3495.601209966525, array([-0.90717683, -0.319823  ]))

In [26]:
get_log_proba([1], data, means, cov, [0, 2, 3])

(-1405.4323728994725, array([ 3.49205536, -0.16036643, -0.04158602]))